In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [48]:
data = pd.read_excel('data/data_ford_price.xlsx')
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [49]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [50]:
x.drop('lat', axis = 1, inplace = True)

Исходная модель

In [51]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


Верно выделены три столбца-признака для обучения, выбранные RFE.

In [52]:
from sklearn.feature_selection import RFE

In [53]:
y = data['price']
x = data.drop(columns='price')

In [54]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [55]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

Обучена регрессия на первых трёх столбцах (RFE), оценено качество модели на тесте.

In [56]:
y = data['price']
x = data.drop(['price', 'odometer', 'long', 'weather'], axis= 1)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
model_RFE = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 5096.570


Верно выделены три столбца-признака для обучения, выбранные SelectKBest.

In [58]:
from sklearn.feature_selection import SelectKBest, f_regression

In [59]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

Обучена регрессия на вторых трёх столбцах (SelectKBest), оценено качество модели на тесте.

In [60]:
y = data['price']
x = data.drop(['price', 'lat', 'long', 'weather'], axis= 1)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
model_RFE = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4708.946


Метод k лучших переменных: SelectKBest показал себя лучше, так как средняя абсолютная ошибка уменьшилась. 